In [73]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import when
from pyspark.sql.functions import col
from pyspark.sql.types import * 
from pyspark.sql.functions import lit
import pyodbc
import pandas as pd
import pyspark.sql.functions as sf
from datetime import date, timedelta
import datetime
from pyspark.sql.window import Window
from pyspark.sql.functions import rank
from pyspark.sql.functions import col
from pyspark.sql.functions import desc, row_number, monotonically_increasing_id
from sqlalchemy import create_engine

In [2]:
spark = SparkSession.builder.config("spark.driver.memory", "15g").getOrCreate()

In [3]:
def generate_time_range_data(startdate,enddate):
    format = '%Y-%m-%d'
    day_time = []
    startdate = datetime.datetime.strptime(startdate, format).date()
    enddate = datetime.datetime.strptime(enddate, format).date()
    delta = enddate - startdate
    for i in range(delta.days + 1):
        day = startdate + timedelta(days=i)
        day = day.strftime('%Y%m%d')
        day_time.append(str(day))
    return day_time

In [6]:
def process_logsearch_t6():
    day_range = generate_time_range_data('2022-06-01','2022-06-14')
    path = "C:\\Users\\ASUS\\OneDrive\\Big_Data_Analytics\\Dataset\\log_search\\"
    df = spark.read.parquet('C:\\Users\\ASUS\\OneDrive\\Big_Data_Analytics\\Dataset\\log_search\\20220601')
    df = df.filter(df.user_id.isNotNull())
    df = df.select('datetime','user_id','keyword')
    result_t6 = df
    for i in day_range[1:]:
        df1 = spark.read.parquet(path+i)
        df1 = df1.filter(df1.user_id.isNotNull())
        df1 = df1.select('datetime','user_id','keyword')
        result_t6 = result_t6.union(df1)
    result_t6 = result_t6.drop(result_t6.datetime)
    result_t6 = result_t6.filter(result_t6.keyword.isNotNull())    
    return result_t6


In [7]:
def process_logsearch_t7():
    day_range = generate_time_range_data('2022-07-01','2022-07-14')
    path = "C:\\Users\\ASUS\\OneDrive\\Big_Data_Analytics\\Dataset\\log_search\\"
    df = spark.read.parquet('C:\\Users\\ASUS\\OneDrive\\Big_Data_Analytics\\Dataset\\log_search\\20220701')
    df = df.filter(df.user_id.isNotNull())
    df = df.select('datetime','user_id','keyword')
    result_t7 = df
    for i in day_range[1:]:
        df1 = spark.read.parquet(path+i)
        df1 = df1.filter(df1.user_id.isNotNull())
        df1 = df1.select('datetime','user_id','keyword')
        result_t7 = result_t7.union(df1)
    result_t7 = result_t7.drop(result_t7.datetime)
    result_t7 = result_t7.filter(result_t7.keyword.isNotNull())
    return result_t7
    
    



In [39]:
def ranking_data(process_result):
    window = Window.partitionBy("user_id").orderBy(col('keyword').desc())
    rank_result = process_result.withColumn('RANK',rank().over(window))
    rank_result = rank_result.filter(rank_result.RANK == '1')
    rank_result = rank_result.distinct()
    return rank_result

In [69]:
def process_most_search(result_t7,result_t6):
    df = spark.read.csv('C:\\Users\\ASUS\\OneDrive\\Big_Data_Analytics\\BigData_Gen2\\Class 6\\HabbitResult',header=True)
    df= df.withColumn('index', row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)
    result_t7 = result_t7.withColumn('index', row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)
    result_t6 = result_t6.withColumn('index', row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)
    df_t7 = df.join(result_t7, df.index == result_t7.index,'inner').drop(result_t7.user_id).drop(result_t7.index).withColumnRenamed('keyword','T7_Keyword')
    df_t6 = df.join(result_t6, df.index == result_t6.index,'inner').drop(result_t6.user_id).drop(result_t6.index).withColumnRenamed('keyword','T6_Keyword')
    result = df_t6.join(df_t7,df_t6.index == df_t7.index,'inner').select(df_t7.Contract,df_t7.Date,df_t7.SportDuration,df_t7.TVDuration,df_t7.ChildDuration,df_t7.RelaxDuration,df_t7.MovieDuration,df_t7.Most_Watch,df_t7.T7_Keyword,df_t6.T6_Keyword)
    return result

In [78]:
def import_to_mysql(result):
    url = 'jdbc:mysql://' + 'localhost' + ':' + '3306' + '/' + 'Data_Engineer'
    driver = "com.mysql.cj.jdbc.Driver"
    user = 'root'
    password = ''
    result = result.withColumnRenamed('Most_Watch','MostWatch')
    result.write.format('jdbc').option('url',url).option('driver',driver).option('dbtable','customer_behaviour').option('user',user).option('password',password).mode('append').save()
    return print("Data Import Successfully")

In [20]:
result_t6 = process_logsearch_t6()

In [21]:
result_t6.show()

+--------+--------------------+
| user_id|             keyword|
+--------+--------------------+
|44887906|            trữ tình|
| 2719170|              bolero|
| 8830996|cậu mang à sĩ hanako|
|41559909|liên minh công lý...|
|49026196|    việt nam vs appa|
|41376437|nhất kiến khuynh tâm|
| 1254139|                giác|
|42534799|            nexsport|
|49190631|Tìm kiếm bằng giọ...|
|91485275|một mảnh băng tâm...|
|45282582|            sáng tạo|
| 2599280|            truc tep|
|49297586|  ngôi nhà náo nhiệt|
|46830828|           trực tiếp|
|45231662|truc tiep bong da...|
|06450121|vn vs afghanistan...|
|49500606|             bóng đá|
|46297858|      tình yêu ấm áp|
|95659318|  lưỡi gươm diệt quỷ|
| 9725680|           kênh vtv6|
+--------+--------------------+
only showing top 20 rows



In [22]:
result_t7 = process_logsearch_t7()

In [23]:
result_t7.show()

+--------+--------------------+
| user_id|             keyword|
+--------+--------------------+
|98331846|    NHUNG NGUOI THUA|
|98424407|               HAPPI|
|06571359|vietnam next top ...|
|47382357| tình yêu từ 0 đến 1|
| 7016618|           phim thái|
|98320714|kẻ trộm mặt trăng...|
|40385123|              sở hữu|
|43952444|          fairy tail|
|41374324|cảnh đẹp ngày vui...|
|93954197|boku no hero acad...|
|06848771|      prime provider|
|95455337|    fairy tail movie|
|49635591|phim 15 năm chờ đ...|
|91154475|               v.i.p|
|49167649|em là thành trì ...|
|90805966|            tế công |
| 4904933|   bằng chứng thép 3|
| 7018473|Tìm kiếm bằng giọ...|
|06524338|bạn trai tôi là h...|
|47758697|sự trỗi dậy của k...|
+--------+--------------------+
only showing top 20 rows



<b> Summary T6 <b>

In [24]:
result_t6.groupBy('keyword').count().orderBy(col('count').desc()).show(truncate=False)


+--------------------------------------------+-----+
|keyword                                     |count|
+--------------------------------------------+-----+
|Liên Minh Công Lý: Phiên bản của Zack Snyder|8741 |
|fairy tail                                  |6771 |
|thiên nga bóng đêm                          |5471 |
|sao băng                                    |4909 |
|nữ thanh tra tài ba                         |4895 |
|bắt ma phá án                               |4837 |
|siêu nhân                                   |3938 |
|mộng hoa lục                                |3411 |
|cuộc chiến thượng lưu                       |3001 |
|cô nàng trong trắng oh woo ri               |2950 |
|vô tình nhặt được tổng tài                  |2646 |
|hội pháp sư                                 |2556 |
|tôi thấy hoa vàng trên cỏ xanh              |2540 |
|why her?                                    |2449 |
|lấy danh nghĩa người nhà                    |2443 |
|bạn trai tôi là hồ ly                       |

<b> Summary T7 <b> 

In [25]:
result_t7.groupBy('keyword').count().orderBy(col('count').desc()).show(truncate=False)


+--------------------------------------------+-----+
|keyword                                     |count|
+--------------------------------------------+-----+
|Liên Minh Công Lý: Phiên bản của Zack Snyder|8774 |
|fairy tail                                  |5596 |
|thanh thanh tử khâm                         |4465 |
|con tim sắt đá                              |4412 |
|chàng hậu                                   |4096 |
|thiên nga bóng đêm                          |3712 |
|siêu nhân                                   |3638 |
|yêu tinh                                    |3423 |
|vẻ đẹp đích thực                            |3394 |
|hội pháp sư                                 |2911 |
|Tìm kiếm bằng giọng nói                     |2837 |
|kẻ trộm mặt trăng: minions                  |2822 |
|nhất kiến khuynh tâm                        |2802 |
|tôi thấy hoa vàng trên cỏ xanh              |2743 |
|Thử Thách Thần Tượng                        |2593 |
|sao băng                                    |

In [40]:
rank_result_t6 = ranking_data(result_t6)

In [41]:
rank_result_t6.show(truncate=False)

+-------+--------------------------------------------+----+
|user_id|keyword                                     |RANK|
+-------+--------------------------------------------+----+
|0000402|xem phim nhan duyen tien dinh thailan       |1   |
|0027835|lời nói dối của hồ ly                       |1   |
|0042335|tình anh quang dũng                         |1   |
|0047402|aq                                          |1   |
|0062340|quân tử lập thân                            |1   |
|0145732|again my life                               |1   |
|0156279|PLUNDERER                                   |1   |
|0164517|lee min ho                                  |1   |
|016773 |Liên Minh Công Lý: Phiên bản của Zack Snyder|1   |
|0201199|vagabond                                    |1   |
|021135 |hồ sơ trinh sát                             |1   |
|0221207|thư ký kim sa                              |1   |
|0258539|animal                                      |1   |
|0286702|đại mạc đao                    

In [42]:
rank_result_t7 = ranking_data(result_t7)

In [44]:
rank_result_t7.show(truncate=False)

+--------+-----------------------------------------------------+----+
|user_id |keyword                                              |RANK|
+--------+-----------------------------------------------------+----+
|0000699 |thả thí                                              |1   |
|0013936 |one pice                                             |1   |
|0027835 |minh lan truyện                                      |1   |
|0077738 |penthouse 3                                          |1   |
|0152453 |d day                                                |1   |
|0162317 |dhoom3                                               |1   |
|0197583 |thap nhi dam                                         |1   |
|0201199 |người thầy y đức 2                                   |1   |
|0209346 |công tố viên chuyển sinh                             |1   |
|0258467 |vẻ đẹp đích thực vietsub                             |1   |
|0276520 |bất động sản Trường Sa                               |1   |
|0292234 |DOREMO    

<b> Testing data <b>

In [45]:
rank_result_t6.filter(rank_result_t6.user_id == '0027835').show(truncate=False)

+-------+---------------------+----+
|user_id|keyword              |RANK|
+-------+---------------------+----+
|0027835|lời nói dối của hồ ly|1   |
+-------+---------------------+----+



In [46]:
rank_result_t7.filter(rank_result_t7.user_id == '0027835').show(truncate=False)

+-------+---------------+----+
|user_id|keyword        |RANK|
+-------+---------------+----+
|0027835|minh lan truyện|1   |
+-------+---------------+----+



<b> Calculating Most search <b>

In [70]:
result = process_most_search(result_t7,result_t6)

In [72]:
result.printSchema()

root
 |-- Contract: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- SportDuration: string (nullable = true)
 |-- TVDuration: string (nullable = true)
 |-- ChildDuration: string (nullable = true)
 |-- RelaxDuration: string (nullable = true)
 |-- MovieDuration: string (nullable = true)
 |-- Most_Watch: string (nullable = true)
 |-- T7_Keyword: string (nullable = true)
 |-- T6_Keyword: string (nullable = true)



In [71]:
result.show()

+---------+----------+-------------+----------+-------------+-------------+-------------+-----------+--------------------+--------------------+
| Contract|      Date|SportDuration|TVDuration|ChildDuration|RelaxDuration|MovieDuration| Most_Watch|          T7_Keyword|          T6_Keyword|
+---------+----------+-------------+----------+-------------+-------------+-------------+-----------+--------------------+--------------------+
|AGAAA0520|2022-04-01|         null|    173458|         null|         null|          447|Truyền Hình|    NHUNG NGUOI THUA|            trữ tình|
|AGAAA0663|2022-04-01|         null|     89001|         null|         null|         null|Truyền Hình|               HAPPI|              bolero|
|AGAAA1058|2022-04-01|         null|    118784|         null|         null|         null|Truyền Hình|vietnam next top ...|cậu mang à sĩ hanako|
|AGAAA1238|2022-04-01|         null|    163158|         null|         null|         null|Truyền Hình| tình yêu từ 0 đến 1|liên minh công

In [79]:
import_to_mysql(result)

Data Import Successfully
